In [1]:
from pathlib import Path
import pandas as pd
from typing import Optional, Dict
from functools import reduce
import geopandas as gpd
import csv
import io, zipfile, requests

### Empirirca regio data

In [2]:
BASE_DIR = Path("..") / "data" / "input" / "empirica_regio_data"

def load_empirica_long(filename: str, region: Optional[str] = None, sheet_name: str = "Daten", value_col: str = "value") -> pd.DataFrame:
    
    path = BASE_DIR / filename

    df = pd.read_excel(path, sheet_name=sheet_name)

    year_cols = [
        c for c in df.columns
        if isinstance(c, (int, float)) or str(c).isdigit()
    ]

    df_long = df.melt(
        id_vars=["Regionsebene", "RegionID", "Regionsname"],
        value_vars=year_cols,
        var_name="Jahr",
        value_name=value_col
    )

    df_long["Jahr"] = df_long["Jahr"].astype(int)
    df_long[value_col] = pd.to_numeric(df_long[value_col], errors="coerce")
    df_long = df_long.sort_values(["RegionID", "Jahr"]).reset_index(drop=True)

    if region is not None:
        df_long = df_long[df_long["Regionsname"] == region].copy()

    return df_long

In [3]:
def load_empirica_multi(files: Dict[str, str], region: Optional[str] = None, sheet_name: str = "Daten") -> pd.DataFrame:

    dfs = []

    for value_col, filename in files.items():
        df = load_empirica_long(
            filename=filename,
            region=region,
            sheet_name=sheet_name,
            value_col=value_col
        )
        dfs.append(df)

    key_cols = ["Regionsebene", "RegionID", "Regionsname", "Jahr"]

    df_merged = reduce(
        lambda left, right: pd.merge(left, right, on=key_cols, how="outer"),
        dfs,
    )
    
    df_merged = df_merged.sort_values(["RegionID", "Jahr"]).reset_index(drop=True)
    
    return df_merged

In [4]:
files = {
    # Kaufpreise (m²-Preise ETW / EZFH, Perzentile)
    "Kaufpreis/m2 ETW 5%":       r"Kaufpreise\Anfangspreise_ETW_Insgesamt.xlsx",
    "Kaufpreis/m2 ETW 50%":      r"Kaufpreise\Standardpreise_ETW_Insgesamt.xlsx",
    "Kaufpreis/m2 ETW 95%":      r"Kaufpreise\Spitzenpreise_ETW_Insgesamt.xlsx",
    "Kaufpreis/m2 EZFH 5%":      r"Kaufpreise\Anfangspreise_EZFH_Insgesamt.xlsx",
    "Kaufpreis/m2 EZFH 50%":     r"Kaufpreise\Standardpreise_EZFH_Insgesamt.xlsx",
    "Kaufpreis/m2 EZFH 95%":     r"Kaufpreise\Spitzenpreise_EZFH_Insgesamt.xlsx",

    # Mieten (m²-Mieten, Perzentile)
    "Mietpreis/m2 5%":               r"Mietpreise\Anfangsmieten_Insgesamt.xlsx",
    "Mietpreis/m2 50%":              r"Mietpreise\Standardmieten_Insgesamt.xlsx",
    "Mietpreis/m2 95%":              r"Mietpreise\Spitzenmieten_Insgesamt.xlsx",

    # Rendite / Finanzierung / Belastung
    "Vervielfältiger":           r"Vervielfältiger.xlsx",
    "Bruttomietrendite":         r"Bruttomietrendite.xlsx",
    "Annuitätenbelastung ETW":   r"Annuitätenbelastung_ETW.xlsx",
    "Annuitätenbelastung EZFH":  r"Annuitätenbelastung_EZFH.xlsx",
    "Mietbelastung":             r"Mietbelastung.xlsx",

    # Demografie / Arbeitsmarkt
    "Einwohner":                 r"Einwohner.xlsx",
    "Durchschnittsalter":        r"Durchschnittsalter.xlsx",
    "Arbeitsvolumen Einwohner":  r"Arbeitsvolumen_Einwohner.xlsx",
    "Arbeitsvolumen Erwerbstätige": r"Arbeitsvolumen_Erwerbstätige.xlsx",
    "Arbeitsvolumen Haushalt":   r"Arbeitsvolumen_Haushalt.xlsx",

    # Einkommen / BIP
    "BIP insgesamt":             r"BIP_Insgesamt.xlsx",
    "Jahreseinkommen Einwohner": r"Jahreseinkommen_Einwohner.xlsx",
    "Jahreseinkommen Haushalt":  r"Jahreseinkommen_Haushalt.xlsx",

    # Wohnen / Wohnflächen / Wohnbestände
    "Wohneigentumsquote":        r"Wohneigentumsquote.xlsx",
    "Genehmigte Wohnungen":      r"Genehmigte_Wohnungen.xlsx",
    "Vermietete Wohnungen":      r"Vermietete_Wohnungen.xlsx",
    "Wohnfläche":                r"Wohnfläche.xlsx",
    "Wohnfläche insgesamt":      r"Wohnfläche_insgesamt.xlsx",
    "Wohnungen":                 r"Wohnungen.xlsx",
}

df_empirica_regio = load_empirica_multi(files)

### Macroeconomic data

In [5]:
BASE_DIR = Path("..") / "data" / "input" / "macroeconomic_data"

# Effektiver Jahreszins
df_immobilienzins = pd.read_csv(BASE_DIR / "Effektiver_Jahreszins.csv", header=None, names=["raw"], encoding="utf-8")
df_immobilienzins = df_immobilienzins["raw"].str.split(";", expand=True)
df_immobilienzins.columns = ["Datum", "Jahr", "Monat", "Effektiver Jahreszins"]
if df_immobilienzins.loc[0, "Datum"].lower().startswith("datum"):
    df_immobilienzins = df_immobilienzins.iloc[1:].reset_index(drop=True)
df_immobilienzins["Datum"] = pd.to_datetime(df_immobilienzins["Datum"])
df_immobilienzins["Jahr"] = df_immobilienzins["Jahr"].astype(int)
df_immobilienzins["Monat"] = df_immobilienzins["Monat"].astype(int)
df_immobilienzins["Effektiver Jahreszins"] = (df_immobilienzins["Effektiver Jahreszins"].astype(str).str.replace(",", ".", regex=False).astype(float))

# S&P500 data
df_SP500 = pd.read_csv(BASE_DIR / "S&P500HistoricalData.csv", sep=";", encoding="utf-8")
df_SP500["Datum"] = pd.to_datetime(df_SP500["Datum"], format="%d.%m.%Y", errors="coerce")
df_SP500["Jahr"] = df_SP500["Datum"].dt.year
df_SP500["Monat"] = df_SP500["Datum"].dt.month
df_SP500["S&P500 Return"] = df_SP500["S&P500 Return"].astype(str).str.replace("%", "", regex=False).str.replace(",", ".", regex=False).astype(float)

# 10Y German Bond
df_10YGermany = pd.read_csv(BASE_DIR / "Y10BondGermany.csv", sep=";")
df_10YGermany["Datum"] = pd.to_datetime(df_10YGermany["Date"], format="%m/%d/%Y", errors="coerce")
df_10YGermany["Jahr"] = df_10YGermany["Datum"].dt.year
df_10YGermany["Monat"] = df_10YGermany["Datum"].dt.month
df_10YGermany = df_10YGermany.drop(columns=["Date", "Open", "High", "Low", "Change%"])

# HICP - Inflation
df_HICP = pd.read_csv(BASE_DIR / "HICP_Inflation.csv", sep=";")
df_HICP["Datum"] = pd.to_datetime(df_HICP["DATE"], format="%Y-%m-%d", errors="coerce")
df_HICP["Jahr"] = df_HICP["Datum"].dt.year
df_HICP["Monat"] = df_HICP["Datum"].dt.month
df_HICP = df_HICP.drop(columns=["DATE", "TIME PERIOD", "Datum"])

# EZB Leitzins
df_Leitzins = pd.read_excel(BASE_DIR / "EZB_Leitzins.xlsx")

# Geschäftsklimaindex
df_Geschäftsklima = pd.read_excel(BASE_DIR / "Geschäftsklimaindex_ifo.xlsx")

# Konsumentenklimaindex (CCI: Consumer Confidence Index)
df_CCI = pd.read_excel(BASE_DIR / "ConsumerConfidenceIndex.xlsx", sheet_name="Daten")

# Zinsbindung (Bauzinsen)
df_Bauzinsen = pd.read_excel(BASE_DIR / "Bauzinsen.xlsx")


In [6]:
df_macroeconomic = (
    df_immobilienzins
        .merge(df_SP500, on=["Datum", "Jahr", "Monat"], how="left")
        .merge(df_10YGermany, on=["Datum", "Jahr", "Monat"], how="left")
        .merge(df_HICP, on=["Jahr", "Monat"], how="left")
        .merge(df_Leitzins, on=["Datum", "Jahr", "Monat"], how="left")
        .merge(df_Geschäftsklima, on=["Jahr", "Monat"], how="left")
        .merge(df_CCI, on=["Jahr", "Monat"], how="left")
        .merge(df_Bauzinsen, on=["Datum", "Jahr", "Monat"], how="left")
)


### Geo data

In [7]:
BASE_GEO = Path("..") / "data" / "input" / "geo_data"
SHAPE_GEM = BASE_GEO / "vg250_01-01.tm32.shape.ebenen" / "vg250_ebenen_0101" / "VG250_GEM.shp"

gdf_gem = gpd.read_file(SHAPE_GEM)
gdf_gem = gdf_gem.to_crs(epsg=4326)
gdf_gem["lon"] = gdf_gem.geometry.centroid.x
gdf_gem["lat"] = gdf_gem.geometry.centroid.y
df_gemeinden_coords = gdf_gem[["AGS", "lat", "lon"]].copy()
df_gemeinden_coords = df_gemeinden_coords.rename(columns={"AGS": "RegionID"})
df_gemeinden_coords["RegionID"] = df_gemeinden_coords["RegionID"].astype("int64")

C:\Users\49176\AppData\Local\Temp\ipykernel_27540\3649422146.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_gem["lon"] = gdf_gem.geometry.centroid.x
C:\Users\49176\AppData\Local\Temp\ipykernel_27540\3649422146.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_gem["lat"] = gdf_gem.geometry.centroid.y


### Save csv

In [8]:
OUTPUT_DIR = Path("..") / "data" / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df_empirica_regio.to_csv(OUTPUT_DIR / "empirica_regio_data.csv", index=False, encoding="utf-8-sig")

df_gemeinden_coords.to_csv(OUTPUT_DIR / "Gemeinden_coordinates.csv", index=False, encoding="utf-8-sig")

df_macroeconomic.to_csv(OUTPUT_DIR / "macroeconomic_data.csv", index=False, encoding="utf-8-sig")

In [10]:
df_macroeconomic

,Datum,Jahr,Monat,Effektiver Jahreszins,S&P500 Return,10YGerman Bond Return,HICP,Einlagefazilität,Hauptrefinanzierungsgeschäft,ifo Geschäftserwartung (Deutschland),ifo Geschäftsklima (Deutschland),ifo Geschäftsklima (Vearbeitendes Gewerbe),ifo Geschäftsklima (Bauhauptgewerbe),CCI,10 Jahre Sollzinsbindung,15 Jahre Sollzinsbindung
0,2003-01-01,2003,1,5.39,-2.74,4.0580,1.1,1.75,2.75,NaN,NaN,-11.0,-48.1,-23.2,NaN,NaN
1,2003-02-01,2003,2,5.18,-1.71,3.8960,1.2,1.75,2.75,NaN,NaN,-9.2,-47.6,-24.5,NaN,NaN
2,2003-03-01,2003,3,5.07,0.84,4.0330,1.2,1.75,2.75,NaN,NaN,-11.1,-45.7,-23.4,NaN,NaN
3,2003-04-01,2003,4,5.04,8.10,4.0650,1.0,1.50,2.50,NaN,NaN,-10.7,-44.9,-21.7,NaN,NaN
4,2003-05-01,2003,5,4.97,5.09,3.6750,0.6,1.50,2.50,NaN,NaN,-11.2,-45.1,-20.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2025-05-01,2025,5,3.70,6.15,2.5090,2.1,2.25,2.40,-13.6,-8.8,-14.2,-18.1,-8.7,3.56,3.69
269,2025-06-01,2025,6,3.72,4.96,2.5970,2.0,2.25,2.40,-10.3,-7.0,-14.2,-15.2,-8.8,3.57,3.70
270,2025-07-01,2025,7,3.72,2.17,2.6930,1.8,2.00,2.15,-9.7,-6.4,-12.1,-14.4,-8.4,3.57,3.69
271,2025-08-01,2025,8,3.76,1.91,2.7222,2.1,2.00,2.15,-8.4,-5.8,-12.5,-15.7,-10.6,3.64,3.82
